In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import operator
import math
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
%matplotlib inline

In [2]:
#df = pd.read_csv("/home/hp/Desktop/College/Sem5/DA/Project/zomato-bangalore-restaurants/zomato.csv")
#Read cleaned Data
mylist = []

for chunk in  pd.read_csv("/home/hp/Desktop/College/Sem5/DA/Project/newzomato.csv", sep=',', chunksize=20000):
    mylist.append(chunk)

big_data = pd.concat(mylist, axis= 0)
df = big_data

del mylist

In [3]:
df.isnull().sum()

name                              0
online_order                      0
book_table                        0
rate                           9970
location                          0
approx_cost(for two people)     314
listed_in(type)                   0
Cuisine1                          0
Cuisine2                          0
rest1                             0
dtype: int64

In [4]:
#Fill na values with median
df['rate'].fillna((df['rate'].median()), inplace=True)
df['approx_cost(for two people)'].fillna((df['approx_cost(for two people)'].median()), inplace=True)

In [5]:
#transform columns i.e factorize the data as there are categorical data
transformed_df = df.copy()
transformed_df['new_Cuisine1'], _ = pd.factorize(transformed_df['Cuisine1'], sort=True)
transformed_df['new_rest1'], _ = pd.factorize(transformed_df['rest1'], sort=True)
transformed_df['new_listed_in'], _ = pd.factorize(transformed_df['listed_in(type)'], sort=True)
transformed_df['new_location'], _ = pd.factorize(transformed_df['location'], sort=True)
transformed_df['new_online_order'], _ = pd.factorize(transformed_df['online_order'], sort=True)
transformed_df['new_book_table'], _ = pd.factorize(transformed_df['book_table'], sort=True)

to_drop = ['Cuisine1',
           'Cuisine2',
           'rest1',
           'listed_in(type)',
           'location',
           'online_order',
           'book_table',
           'name']
transformed_df.drop(to_drop, inplace=True, axis=1)
transformed_df.dropna(how='any', inplace=True)

In [6]:
train, test = train_test_split(transformed_df, test_size=0.2)

In [7]:
train_x = train.loc[:, train.columns != 'rate']
train_y = train.loc[:, train.columns == 'rate']
test_x = test.loc[:, test.columns != 'rate']
test_y = test.loc[:, test.columns == 'rate']

In [8]:
#decision tree regressor model
regressor = DecisionTreeRegressor(random_state = 0)

In [9]:
regressor.fit(train_x, train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [10]:
y_pred = list(regressor.predict(test_x))

In [11]:
#Metric for the built model
y_pred = np.around(y_pred,decimals=1)
test_y = list(test_y['rate'])
y_pred = list(y_pred)
acc = 0
for i in range(len(y_pred)):
    if(round((y_pred[i] - test_y[i]),1)<0.1):
        acc = acc + 1

print(acc/len(y_pred))

0.7420304070622854


In [12]:
regressor.feature_importances_

array([0.22370553, 0.13781938, 0.09022866, 0.04519891, 0.24831713,
       0.04339988, 0.2113305 ])